In [ ]:
#knock70
import pandas as pd
import pickle
import string
import torch
import gensim.downloader as api
import pickle

# Google News Word2Vecモデルのダウンロード
model = api.load('word2vec-google-news-300')

# モデルをpickleで保存
with open("word2vec.pkl", "wb") as f:
    pickle.dump(model, f)

def trans_word2vec(text):
    table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # 句読点をスペースに置換
    words = text.translate(table).split()  # スペースで分割
    vec = [model[word] for word in words if word in model]  # モデルにある単語をベクトルに変換

    if vec:
        return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをテンソルとして返す
    else:
        return torch.zeros(model.vector_size)  # モデルに単語がない場合はゼロベクトルを返す

with open("word2vec.pkl", "rb") as f:
    model = pickle.load(f)


train = pd.read_csv("train.txt", sep='\t')
valid = pd.read_csv("valid.txt", sep='\t')
test = pd.read_csv("test.txt", sep='\t')

#各データセットのタイトルをtrans_word2vec関数を使用してWord2Vecベクトルに変換したものを、PyTorchテンソルにスタック
X_train = torch.stack([trans_word2vec(text) for text in train['TITLE']])
X_valid = torch.stack([trans_word2vec(text) for text in valid['TITLE']])
X_test = torch.stack([trans_word2vec(text) for text in test['TITLE']])

#カテゴリ（'b'、't'、'e'、'm'）を数値（0、1、2、3）にマッピング
category = {'b': 0, 't': 1, 'e': 2, 'm': 3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category[x]).values)

#Word2VecベクトルとカテゴリラベルをPyTorchテンソルとして保存
torch.save(X_train, 'X_train.pt')
torch.save(X_valid, 'X_valid.pt')
torch.save(X_test, 'X_test.pt')
torch.save(y_train, 'y_train.pt')
torch.save(y_valid, 'y_valid.pt')
torch.save(y_test, 'y_test.pt')
